# [모듈 1.1] 패키징: 서비스 카탈로그 포트폴리오 및 프러덕트
## 부제: Package CloudFormation Templates

노트북은 필수 리소스를 생성하기 위한 다음 활동을 안내합니다.

1. 사전 참고
2. S3 버킷 설정
3. 소스 코드 준비
    - 3.1. Source code zip file 생성
    - 3.2 소스 코드 zip 파일을 S3 버킷에 업로드
4. package-cfn.yaml 배포 파일 확인
5. package-cfn.yaml (CloudFormation 템플릿) 을 배포
    - 최종적으로 아래와 같은 파일을 S3에 업로딩 합니다.
        - amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml
            -  카탈로그 포트폴리오, 프러덕트 생성 YAML 파일 
        - amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip
            - 카탈로그 포트폴리오, 프러덕트 관련된 파일을 압축 함.
        - amazon-sagemaker-reusable-components/project-s3-fs-ingestion.yaml
            - 세이지 메이커 파이프라인 프로젝트 생성 YAML 파일
        - amazon-sagemaker-reusable-components/seed-code/s3-fs-ingestion-v1.0.zip
            - 세이지 메이커 파이프라인 프로젝트 생성시에 CodeCommit에 등록될 소스 코드
        
---

# 1. 사전 참고

## 1.1 참고: 코드 수정 (원문 대비 변경 사항. 확인하시고 스킵하세요.)
- cfn-templates/sm-project-sc-portfolio.yaml 파일을 아래와 같이 수정 함.
    - Amazon SageMaker execution role 이름을 얻는데 에러가 발생을 하여 수정

Before:
```YAML
AmazonSageMakerExecutionRoleName:
    Description: Name of the Amazon SageMaker execution role
    Value: !Select [2, !Split ['/', !Ref SCPortfolioPrincipalRoleArn ] ] 
```
After:
```YAML
  AmazonSageMakerExecutionRoleName:
    Description: Name of the Amazon SageMaker execution role
    Value: !Select [1, !Split ['/', !Ref SCPortfolioPrincipalRoleArn ] ] 
```


## 2. S3 버킷 설정
- 디폴트 버킷을 사용 합니다. 사용자 정의 버킷을 사용시에는 아래 코드를 수정해서 입력 해주시기 바랍니다.

In [1]:
import sagemaker
Use_Custome_Bucket = None
if Use_Custome_Bucket is None:
    bucket = sagemaker.Session().default_bucket()
    bucket_region = sagemaker.Session().boto_region_name
else:    
    bucket = '<>' # 사용자 버킷을 이용하세요.
    bucket_region = '<>' # 위의 버킷 리젼을 입력 하세요.

print("bucket: ", bucket)
print("bucket_region: ", bucket_region)

bucket:  sagemaker-us-east-1-569441333767
bucket_region:  us-east-1


# 3. 소스 코드 준비

## 3.1. Source code zip file 생성

- 제공된 Git 리파지토리에서 yaml, yml, sh, py 파일을 amazon-sagemaker-reusable-components.zip 로 압축하여 소스 코드를 준비 합니다.
원문을 아래와 같이 `*.py` 를 추가 했습니다. 이렇지 않은 경우 에러 발생 합니다.
- 참고:
    - 원문: zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh"
    - 수정: zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh" "*.py"

In [2]:
%%sh

# cd amazon-sagemaker-reusable-components 
rm -f amazon-sagemaker-reusable-components.zip
# zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh"
zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh" "*.py"

  adding: package-cfn.sh (deflated 66%)
  adding: sm-project-sc-portfolio.yaml (deflated 77%)
  adding: buildspec-package-cfn.yml (deflated 18%)
  adding: package-cfn.yaml (deflated 69%)
  adding: project-seed-code/s3-fs-ingestion/build.py (deflated 63%)
  adding: project-seed-code/s3-fs-ingestion/pipeline/pipeline.py (deflated 70%)
  adding: project-seed-code/s3-fs-ingestion/functions/start_fs_ingestion.py (deflated 58%)
  adding: project-seed-code/s3-fs-ingestion/buildspec.yml (deflated 54%)
  adding: prerequsite/create-sm-notebook.yaml (deflated 59%)
  adding: cfn-templates/project-s3-fs-ingestion.yaml (deflated 72%)
  adding: cfn-templates/sm-project-sc-portfolio.yaml (deflated 76%)
  adding: cfn-templates/.ipynb_checkpoints/sm-project-sc-portfolio-checkpoint.yaml (deflated 76%)


## 3.2 소스 코드 zip 파일을 S3 버킷에 업로드
- 소스 코드 zip 파일 `amazon-sagemaker-reusable-components.zip`을 S3 버킷에 업로드합니다.

In [3]:
# S3_BUCKET_NAME=<your existing or just created S3 bucket name>
! aws s3 cp amazon-sagemaker-reusable-components.zip s3://{bucket}/amazon-sagemaker-reusable-components/

upload: ./amazon-sagemaker-reusable-components.zip to s3://sagemaker-us-east-1-569441333767/amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip


In [4]:
! aws s3 ls {bucket} --recursive
# ! aws s3 rm s3://{bucket} --recursive

2022-02-07 01:35:49          0 569441333767/sagemaker/us-east-1/offline-store/FG-abalone-07-01-25-47-c99ecc2d-1644197747/FG-abalone-07-01-25-47-c99ecc2d2022-02-07T01:35:47.967Z.txt
2022-02-07 03:13:29      19868 amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip
2022-02-07 01:08:51      15598 amazon-sagemaker-reusable-components/project-s3-fs-ingestion.yaml
2022-02-07 01:08:49       4326 amazon-sagemaker-reusable-components/seed-code/s3-fs-ingestion-v1.0.zip
2022-02-07 01:08:50       7205 amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml


# 4. package-cfn.yaml 배포 파일 확인

### 4.1 CF 의 Output 확인
- SMProjectSCPortfolioS3Uri 생성
- SMProjectSCPortfolioDeployLink 생성
- StartBuildCLICommand 생성

In [5]:
!pygmentize "package-cfn.yaml"   | sed -n 13,24p

Outputs:
  SMProjectSCPortfolioS3Uri:
    Description: S3 URI for SageMaker projects as Service Catalog portfolio deployment stack
    Value: !Sub 'https://s3.${AWS::Region}.amazonaws.com/${S3BucketName}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml'

  SMProjectSCPortfolioDeployLink:
    Description: Link to open CloudFormation deployment of Service Catalog portfolio with SageMaker projects
    Value: !Sub 'https://console.aws.amazon.com/cloudformation/home?region=${AWS::Region}#/stacks/new?templateURL=https://s3.${AWS::Region}.amazonaws.com/${S3BucketName}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml'

  StartBuildCLICommand:
    Description: CLI to start CodeBuild build
    Value: !Sub 'aws codebuild start-build --project-name ${CfnTemplatePackageProject}'


### 4.2 리소스 확인

#### StartBuildLambdaExecutionRole 생성
- 람다 함수(예: amazon-sagemaker-reusable-compone-StartBuildLambda-FI8vkTm6S8AT) 를 실행하는 역할 생성


In [6]:
!pygmentize "package-cfn.yaml"   | sed -n 26,52p

Resources:

  StartBuildLambdaExecutionRole:  
    Type: 'AWS::IAM::Role'
    Properties:
      AssumeRolePolicyDocument:
        Version: 2012-10-17
        Statement:
          - Effect: Allow
            Principal:
              Service:
                - lambda.amazonaws.com
            Action:
              - 'sts:AssumeRole'
      Path: /
      Policies:
        - PolicyName: InlinePolicy
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Sid: CodeBuildPermission
                Effect: Allow
                Action:
                  - codebuild:StartBuild
                Resource: !GetAtt CfnTemplatePackageProject.Arn     
      ManagedPolicyArns:
        - 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'


### 4.3. StartBuildLambda 함수 생성
- `cb.start_build(projectName=event['ResourceProperties']['ProjectName'])` 실행

In [7]:
!pygmentize "package-cfn.yaml"   | sed -n 54,86p

  StartBuildLambda:
    Type: AWS::Lambda::Function
    Properties:
      ReservedConcurrentExecutions: 1
      Code:
        ZipFile: |
          # Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
          # SPDX-License-Identifier: MIT-0
          import json
          import boto3
          import cfnresponse

          cb = boto3.client("codebuild")

          def lambda_handler(event, context):
              try:
                  response_status = cfnresponse.SUCCESS

                  if 'RequestType' in event and event['RequestType'] == 'Create':
                      cb.start_build(projectName=event['ResourceProperties']['ProjectName'])
                      
                  cfnresponse.send(event, context, response_status, {}, '')

              except Exception as e:
                  print(str(e))
                  cfnresponse.send(event, context, cfnresponse.FAILED, {}, physicalResourceId=event.get('PhysicalResourceId'), reason=str(e))

      Descripti

### 4.4. StartBuild 및 CodeBuildServiceRole 생성
- CodeBuild 를 위한 커스텀 StartBuild 및 역할 생성

In [8]:
!pygmentize "package-cfn.yaml"   | sed -n 88,139p

  StartBuild:
    Type: Custom::StartBuild
    Properties:
      ServiceToken: !GetAtt StartBuildLambda.Arn
      ProjectName: !Ref CfnTemplatePackageProject

  CodeBuildServiceRole:
    Type: 'AWS::IAM::Role'
    Properties:
      AssumeRolePolicyDocument:
        Version: 2012-10-17
        Statement:
          - Effect: Allow
            Principal:
              Service: codebuild.amazonaws.com
            Action: 'sts:AssumeRole'
      Path: '/service-role/'
      Policies:
        - PolicyName: CodeBuildServiceRoleInLinePolicy
          PolicyDocument:
            Version: 2012-10-17
            Statement:
              -
                Action:
                  - 'logs:CreateLogGroup'
                  - 'logs:CreateLogStream'
                  - 'logs:PutLogEvents'
                Resource: '*'
                Effect: 'Allow'
              -
                Action:
                  - 's3:*Object'
                  - 's3:GetObjectVersion'
                  - 's3:GetBucketAcl'
 

### 4.5. CfnTemplatePackageProject 생성
- 이 빌드 프로젝트는 buildspec-package-cfn.yml 실행 함.

In [9]:
!pygmentize "package-cfn.yaml"   | sed -n 140,163p

  CfnTemplatePackageProject:
    Type: AWS::CodeBuild::Project
    Properties:
      Description: !Sub 'Packaging CFN templates into ${S3BucketName}'
      ServiceRole: !GetAtt CodeBuildServiceRole.Arn
      Artifacts: 
        Type: NO_ARTIFACTS
      Environment:
        Type: 'LINUX_CONTAINER'
        ComputeType: 'BUILD_GENERAL1_SMALL'
        Image: 'aws/codebuild/amazonlinux2-x86_64-standard:3.0'
        EnvironmentVariables:
          - Name: 'S3_BUCKET_NAME'
            Value: !Ref S3BucketName
          - Name: 'DEPLOYMENT_REGION'
            Value: !Ref AWS::Region
      Source:
        Type: S3
        Location: !Sub '${S3BucketName}/amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip'
        BuildSpec: buildspec-package-cfn.yml
      LogsConfig:
        CloudWatchLogs:
          Status: 'ENABLED'
      TimeoutInMinutes: 5


### 4.6. buildspec-package-cfn.yml 확인
```
./package-cfn.sh ${S3_BUCKET_NAME} ${DEPLOYMENT_REGION} 실행 함
```

In [10]:
!pygmentize "buildspec-package-cfn.yml" 

version: 0.2

phases:
  build:
    commands:
      - env
      - echo Packaging Cloudformation and uploading to S3...
      - ./package-cfn.sh ${S3_BUCKET_NAME} ${DEPLOYMENT_REGION}


### 4.7. package-cfn.sh 확인
```
./package-cfn.sh ${S3_BUCKET_NAME} ${DEPLOYMENT_REGION} 실행 함
```

- 이 셀은  아래의 주요한 파일을 Zip 으로 압축하고 S3에 업로딩 합니다.
    - (1) sm-project-sc-portfolio.yaml (카탈로그 포트폴리오 정의 파일)
    - (2) project-s3-fs-ingestion.yaml (카탈로그 프러덕트 정의 파일)
    - (3) 카탈로그 프러덕트 Seed 코드 (Pipeline 정의 등) 추가
    - (4) 지정된 S3 버킷에 위의 결과 파일을 업로드


- 아래 코드를 스키밍 하시고 실행 결과는 추후에 확인 함.

In [11]:
!pygmentize "package-cfn.sh" 

#!/bin/bash

# Copyright 2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

set -e

# This script will package the CloudFormation in ${CFN_TEMPLATE_DIR} directory and upload it 
# to Amazon S3 in preparation for deployment using the AWS CloudFromation service.  
# 
# This script exists because Service Catalog products, when using relative references to cloudformation templates are 
# not properly packaged by the AWS cli. Also the full stack, due to 2 levels of Service Catalog deployment will not 
# always package properly using the AWS cli.

# This script treats the templates as source code and packages them, putting the results into a 'build' subdirectory.

# This script assumes a Linux or MacOSX environment and relies on the following software packages being installed:
# . - AWS Command Line Interface (CLI)
# . - sed
# . - Python 3 / pip3
# . - zip

# PLEASE NOTE this script will store all resources to an Amazon S3 bucket s3://${CFN_BUCKET

# 5. package-cfn.yaml (CloudFormation 템플릿) 을 배포
- sm-project-sc-portfolio.yaml 생성
- seed-code/s3-fs-ingestion-v1.0.zip 생성

## 5.1. 클라우드 포메이션 실행
- 약 3분 정도 소요 됩니다.
- Cloud Formation 콘솔로 가서 진행 사항을 확인할 수 있습니다.

In [12]:
# Deploy the CloudFormation template:
STACK_NAME='amazon-sagemaker-reusable-components-package-cfn'
! aws cloudformation deploy \
        --template-file package-cfn.yaml \
        --stack-name {STACK_NAME} \
        --capabilities CAPABILITY_NAMED_IAM \
        --parameter-overrides \
        S3BucketName={bucket}


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - amazon-sagemaker-reusable-components-package-cfn


In [13]:
! aws cloudformation describe-stacks \
    --stack-name {STACK_NAME} \
    --output table \
    --query "Stacks[0].Outputs[*].[OutputKey, OutputValue]"

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|                                                                                                                            DescribeStacks                                                                                                                             |
+--------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|  SMProjectSCPortfolioS3Uri     |  https://s3.us-east-1.amazonaws.com/sagemaker-us-east-1-569441333767/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml                                 

## 5.2. CloudFormation 템플릿 실행 결과
- 두개의 중요한 Output 이 출력 됩니다.
    - 카탈로그 포트폴리오 정의하는 CF: `sm-project-sc-portfolio.yaml` 
    - 카탈로그 포트폴리오 정의 링크:  SMProjectSCPortfolioDeployLink

예시: 
```
|  SMProjectSCPortfolioS3Uri     |  https://s3.us-east-1.amazonaws.com/sagemaker-us-east-1-569441333767/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml                                                                                               |
|  SMProjectSCPortfolioDeployLink|  https://console.aws.amazon.com/cloudformation/home?region=us-east-1#/stacks/new?templateURL=https://s3.us-east-1.amazonaws.com/sagemaker-us-east-1-569441333767/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml   |
|  StartBuildCLICommand          |  aws codebuild start-build --project-name CfnTemplatePackageProject-ZGoGzOIfOy4h  
+--------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
```

## 5.2. package-cfn.yaml (CloudFormation 템플릿) 실행 과정
- 아래와 같이 CodeBuild (예: "CfnTemplatePackageProject-acyQl0hj9nBO") 코드 빌드 프로젝트가 실행이 되었습니다.

![cfn-template.png](img2/cfn-template.png)

## 5.3. "package-cfn.sh" 실행 결과
- 위의 CodeBuild는 실질적으로 "package-cfn.sh" 를 실행 하였습니다.
- 아래의 로그는 다음과 같이 Zip 파일 생성 작업을 합니다.
    - (1) sm-project-sc-portfolio.yaml (카탈로그 포트폴리오 정의 파일) 
    - (2) project-s3-fs-ingestion.yaml (카탈로그 프러덕트 정의 파일)
    - (3) 카탈로그 프러덕트 Seed 코드 (Pipeline 정의 등) 추가

```
[Container] 2022/02/04 05:49:26 Running command ./package-cfn.sh ${S3_BUCKET_NAME} ${DEPLOYMENT_REGION}
127	Preparing content for publication to Amazon S3 s3://sagemaker-ap-northeast-2-XXXXXXXXX/amazon-sagemaker-reusable-components
128	Zipping the CloudFormation templates and buildspec files
129	  adding: build/ap-northeast-2/project-s3-fs-ingestion.yaml (deflated 72%)
130	  adding: build/ap-northeast-2/sm-project-sc-portfolio.yaml (deflated 76%)
131	  adding: package-cfn.sh (deflated 66%)
132	  adding: buildspec-package-cfn.yml (deflated 18%)
133	  adding: cfn-templates/project-s3-fs-ingestion.yaml (deflated 72%)
134	  adding: cfn-templates/.ipynb_checkpoints/sm-project-sc-portfolio-checkpoint.yaml (deflated 76%)
135	  adding: cfn-templates/sm-project-sc-portfolio.yaml (deflated 76%)
136	  adding: .ipynb_checkpoints/package-cfn-checkpoint.yaml (deflated 69%)
137	  adding: .ipynb_checkpoints/buildspec-package-cfn-checkpoint.yml (deflated 18%)
138	  adding: project-seed-code/s3-fs-ingestion/buildspec.yml (deflated 54%)
139	  adding: package-cfn.yaml (deflated 69%)
140	Zipping MLOps project seed code
141	  adding: pipeline/ (stored 0%)
142	  adding: pipeline/pipeline.py (deflated 70%)
143	  adding: build.py (deflated 63%)
144	  adding: functions/ (stored 0%)
145	  adding: functions/start_fs_ingestion.py (deflated 58%)
146	  adding: buildspec.yml (deflated 54%)
147	Publishing CloudFormation to ap-northeast-2
```

![cfn-template-build-log.png](img2/cfn-template-build-log.png)

## 5.4. S3에 생성된 파일 확인

In [14]:
! aws s3 ls s3://{bucket}/amazon-sagemaker-reusable-components/ --recursive

2022-02-07 03:13:29      19868 amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip
2022-02-07 01:08:51      15598 amazon-sagemaker-reusable-components/project-s3-fs-ingestion.yaml
2022-02-07 01:08:49       4326 amazon-sagemaker-reusable-components/seed-code/s3-fs-ingestion-v1.0.zip
2022-02-07 01:08:50       7205 amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml


# 6. 변수 저장
- 다음 노트북에서 사용할 변수 저장

In [15]:
%store bucket
%store bucket_region

Stored 'bucket' (str)
Stored 'bucket_region' (str)


# 7. Next
- 다음 노트북 "2.1 Deploy-Catalog-Portfolio-Product.ipynb" 로 이동 하세요.

---
# A. 트러블 슈팅 커맨드 및 에러 케이스

## A.1. 클라우드 포메이션 스택 진행 상황, 삭제 및 기타 명령어

In [ ]:
# 스택 이벤트 실행 내역 보여주기
! aws cloudformation describe-stack-events --stack-name amazon-sagemaker-reusable-components-package-cfn

In [ ]:
# 스택 삭제
! aws cloudformation delete-stack --stack-name amazon-sagemaker-reusable-components-package-cfn

## A.2 S3 CLI 관련 명령어

In [ ]:
! aws s3 ls https://s3.{bucket_region}.amazonaws.com/{bucket}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml
# ! aws s3 ls https://s3.ap-southeast-2.amazonaws.com/sagemaker-ap-southeast-2-057716757052/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml

In [ ]:
! aws s3 ls {bucket} --recursive

In [ ]:
! aws s3 rm s3://{bucket} --recursive

## A.3. 클라우드 포메이션 실행시 권한 에러
- 아래와 같은 필수 권한을 세이지 메이커 노트북을 실행하는 역할에 추가 해주세요.
    - "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"
    - "arn:aws:iam::aws:policy/AmazonS3FullAccess"
    - "arn:aws:iam::aws:policy/IAMFullAccess"        
    - "arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess"        
    - "arn:aws:iam::aws:policy/AWSServiceCatalogAdminFullAccess"
    - "arn:aws:iam::aws:policy/AWSCloudFormationFullAccess"
    - "arn:aws:iam::aws:policy/AWSLambda_FullAccess"




## A.4. 에러: seed-code/s3-fs-ingestion-v1.0.zip 생성시 전부 아티펙트를 포함하지 않음.
```
Zipping MLOps project seed code
147	  adding: .ipynb_checkpoints/ (stored 0%)
148	  adding: .ipynb_checkpoints/buildspec-checkpoint.yml (deflated 54%)
149	  adding: buildspec.yml (deflated 54%)
```
정상 케이스
```
Zipping MLOps project seed code
  adding: LICENSE (deflated 39%)
  adding: pipeline/ (stored 0%)
  adding: pipeline/pipeline.py (deflated 70%)
  adding: buildspec.yml (deflated 54%)
  adding: .ipynb_checkpoints/ (stored 0%)
  adding: .ipynb_checkpoints/buildspec-checkpoint.yml (deflated 54%)
  adding: build.py (deflated 63%)
  adding: README.md (stored 0%)
  adding: functions/ (stored 0%)
  adding: functions/start_fs_ingestion.py (deflated 58%)
```

### 해결
- Before
    - zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh"
- After
    - zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh" "*.py"